IMPORTING LIBRARIES

In [1]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
from transformers import pipeline

LOADING THE CONTEXT FOR REFERNCE TO QUESTIONS 

In [2]:
documents = [
    "To track your Amazon order, log into your account, go to 'Your Orders,' and click 'Track Package' for real-time updates.",
    "Amazon's return policy allows most items to be returned within 30 days of delivery for a full refund, provided they are in new condition with original packaging and accessories.",
    "To return an Amazon order, initiate a return through 'Your Orders,' ship the item back, and receive a refund once processed.",
    "To contact Amazon customer service, use the 'Help' section on the website or app to chat, call, or email support.",
    "Amazon Prime members receive free two-day shipping, exclusive deals, and access to Prime Video and Music.",
    "If your Amazon package is delayed, check the estimated delivery date in 'Your Orders' or contact customer service for assistance.",
    "To cancel an Amazon order, go to 'Your Orders,' select the order, and click 'Cancel Items' if it hasn't shipped yet.",
    "To purchase an Amazon gift card, visit the Amazon website, navigate to 'Gift Cards,' select a design and amount, add to cart, and complete the purchase at checkout; the gift card can be redeemed for eligible products.",
    "To update your Amazon payment method, go to 'Your Account,' select 'Your Payments,' and add or edit your card details.",
    "To log into your Amazon account, go to the Amazon website or app, click 'Sign In,' and enter your email or phone number and password."
]

TEXT EMBEDDING TO CONVERT IT INTO VECTORS

In [ ]:
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
doc_embedding = embedding_model.encode(documents)
doc_embedding = np.array(doc_embedding).astype('float32')

SIMILAARITY SEARCH USING FAISS

In [4]:
dimension = doc_embedding.shape[1]
index = faiss.IndexFlatIP(dimension)
faiss.normalize_L2(doc_embedding)
index.add(doc_embedding)

CREATING PIPELINE

In [ ]:
generator = pipeline('text-generation',model='google/flan-t5-small',tokenizer='google/flan-t5-small')

RAG ANSWER FUNCTION

In [12]:
def rag_answer(query, top_k=3, threshold = 0.2):
    query_embedding = embedding_model.encode([query]).astype('float32')
    faiss.normalize_L2(query_embedding)
    
    distance, indices = index.search(query_embedding,top_k)
    
    if distance[0][0] < threshold:
        return None, "sorry, I couldn't find right answer "
    
    retrived_doc = documents[indices[0][0]]
    
    prompt = f"""
    context: {retrived_doc}
    Question: {query}
    Answer in one sentence: """
    response = generator(
        prompt,
        max_new_tokens=60,
        do_sample=False,
        temperature=0.0
    )[0]['generated_text']

    
    return retrived_doc, response

In [ ]:
def Q_A_Bot():
    print('Hello welcome to Q&A bot')
    
    while True:
        query = input('user: ')
        if query.lower()=='exit':
            print('Goodbye')
            break
        if not query.strip():
            print('please enter valid question')
            continue
        context,answer = rag_answer(query)
        print(f"Helpbot: {answer}\n")
Q_A_Bot()
        